## Projet d'optimisation ##

In [1]:
import numpy as np
from scipy import optimize
from casadi import *
import time

# Question 1

Le but du problème est de trouver l'allocation globale $x \in \mathbb{R}^m$ qui minimise le risque associé au rendement du portefeuille, i.e. la variance de cette allocation par rapport à la variation du prix des actifs $p \in \mathbb{R}^m$.

La variance est l'équivalent d'un "écart à la moyenne" ; elle représente donc bien un indicateur du risque associé à un investissement. Ce risque prend en compte le lien entre la variation des prix et les allocations, d'où $x^T \Sigma x$, où $\Sigma$ est la matrice de covariance associé à $p$.

La relation $\bold{1}^T x = 1$ signifie simplement que l'on normalise l'allocation par souci de cohérence et de clarté ; pour une allocation donnée, le rendement représente le gain moyen associé à l'allocation, d'où $\bar p^T x = r$.

# Question 2

On souhaite minimiser le risque, donc on a
$$
\begin{array}{rcl}
f : z & \longmapsto & f(z) = x^T \Sigma x
\end{array}
$$

La variable $z$ dépend donc de $x$, mais aussi de $\Sigma = \left[ \mathrm{Cov}(p_i, p_j) \right]_{i,j \in \llbracket 1, m \rrbracket}$ et donc de $p$. Le nombre de variables de décisions est donc $n = 2$, où chaque variable est un vecteur de $\mathbb{R}^m$.

L'énoncé contraint le problème par le biais de la condition de normalisation $\bold{1}^T x = 1$ et celle de rendement $r = \bar p^T x$ : on peut donc définir $$c_{eq,norm}(z) = \bold{1}^T x - 1$$ et $$c_{eq,rend}(z) = \bar p^T x - r$$ ce qui ajoute les contraintes égalité $c_{eq} : \begin{cases} c_{eq,norm}(z) = 0 \\ c_{eq,rend}(z) = 0 \end{cases}$ au problème de minimisation de $f$.

# Question 3

On ajoute une contrainte inégalité pour limiter les positions _short_ à un certain montant par allocation, afin de garantir un équilibre de marché, ne pas entraîner de trop fortes fluctuation des actifs et donc une variance trop élevée.

On a donc dans ce problème une contrainte égalité (représentée par la fonction $c_{eq}$) et une contrainte inégalité.

Or la contrainte inégalité comprend déjà une fonction $max$ ; on cherche donc à minimiser une fonction $f$ sous contrainte d'une limitation d'un maximum, ce qui risque de compliquer les calculs...

# Question 4

$\max(-x,\bold{0})$ vérifie
$\forall x \in \mathbb{R}^m, \max(-x,\bold{0}) ≥ -x$ et $\max(-x,\bold{0}) ≥ 0$.

En introduisant $s \in \mathbb{R}^m$ tel que $s ≥ -x$, $s ≥ 0$ et $\bold{1}^T s ≤ s_M$, on a 
$$\bold{1}^T \max(-x,\bold{0}) ≤ \bold{1}^T s ≤ s_M$$
En prenant $s$ le plus proche possible de $\max(-x,\bold{0})$, on peut reformuler de manière équivalente le problème de la question 2 :
$$\min f(z)$$
sous les contraintes $c_{eq}(z) = 0$, $c_{in}(s) ≤ 0$ 

avec $\begin{cases} s ≥ -x \\ s ≥ 0 \end{cases}$ et $c_{in}(s) = \bold{1}^T s - s_M$

La variable $s$ est indirectement liée à $x$, et donc à $z$, puisqu'elle est choisie en fonction de $-x$ (on veut avoir $s = \max(-x, \bold{0})$ en se débarrassant du max).

# Question 5

Le problème $(4)$ est un problème d'optimisation sous contrainte avec une contrainte égalité affine.

Une première méthode de résolution est d'utiliser les multiplicateurs de Lagrange, en calculant le lagrangien
$$\mathcal{L}(z, \lambda) = f(z) + \lambda^T c_{eq}(z)$$
puis en  trouvant ses points stationnaires.

On peut aussi penser aux conditions $(KKT)$ : en effet, une contrainte affine étant convexe, on a une équivalence entre respecter les conditions $(KKT)$ et être solution du problème d'optimisation sous contrainte affine.

# Question 6

Pour résoudre numériquement le problème, on va utiliser les bibliothèques Scipy (avec son module _optimize_ et son solveur SLSQP) et Casadi (avec _ipopt_).

In [2]:
# Définition des variables
p1 = 0.05
p2 = 0.15
p3 = 0.3
p = np.array([p1, p2, p3])
rho = 0.1
r = 0.1

sig1 = 0.1
sig2 = 0.3
sig3 = 0.8
sigma = np.array([[sig1**2, rho*sig1*sig2, 0], [rho*sig1*sig2, sig2**2, 0], [0, 0, sig3**2]])

unit = np.array([1., 1., 1.])
x0 = np.array([0.0, 0.0, 0.0])
x_min = np.zeros_like(x0)

In [3]:
# Définition de la fonction f et des contraintes

def var(x):
    return x.T @ sigma @ x

contraintes = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r})

In [4]:
# BFGS avec Scipy
print('méthode SLSQP avec Scipy (r = 0.1)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes, options={'disp': True})
print("Durée : ", time.time()-start_time)
print("Erreur finale :", np.linalg.norm(res.x-x_min))

méthode SLSQP avec Scipy (r = 0.1)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.017067775457853502
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Durée :  0.0018661022186279297
Erreur finale : 0.6971018902567288


In [5]:
# Résolution avec Casadi
opti_cons = casadi.Opti();
x = opti_cons.variable(3)

opti_cons.minimize(var(x))
opti_cons.subject_to(dot(unit.T, x) - 1 == 0)
opti_cons.subject_to(dot(p.T, x) - r == 0)

opti_cons.set_initial(x,x0)
opti_cons.solver('ipopt')

print("Résolution avec Casadi...")
start_time = time.time()
sol = opti_cons.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x))

Résolution avec Casadi...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
To

On remarque que le temps de calcul avec Scipy et Casadi sont sensiblement les mêmes (de l'ordre de $10^{-3}$ s)

De plus, les 2 méthodes aboutissent au même résultat, à savoir une variance de l'ordre de $10^{-2}$.

# Question 7

$(a)$

$\rho$ est le coefficient de corrélation associé aux vecteurs $p_1$ et $p_2$. Il peut prendre des valeurs dans l'intervalle $[-1, 1]$.

$(b)$

In [6]:
# définition des variables


$(c)$

# Question 8

$(a)$

On ne résout qu'avec la méthode SLSQP de Scipy à présent pour plus de clarté, on n'utilise plus Casadi mais la réponse reste la même (cf. codes masqués ci-dessous).

In [7]:
# Pour r = 0.2
r1 = 0.2

contraintes1 = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r1})

print('  Méthode SLSQP avec Scipy (r = 0.2)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes1, options={'disp': True})
print("Durée : ", time.time()-start_time)
print("Erreur finale :", np.linalg.norm(res.x-x_min))

  Méthode SLSQP avec Scipy (r = 0.2)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.10745197524851792
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Durée :  0.001753091812133789
Erreur finale : 0.8553154235324663


In [8]:
# pour r = 0.15
r2 = 0.15

contraintes2 = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r2})

print('  Méthode SLSQP avec Scipy (r = 0.15)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes2, options={'disp': True})
print("Durée : ", time.time()-start_time)
print("Erreur finale :", np.linalg.norm(res.x-x_min))

  Méthode SLSQP avec Scipy (r = 0.15)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.04956735987023322
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Durée :  0.0017039775848388672
Erreur finale : 0.634496924210732


In [9]:
'''
# équivalent avec Casadi pour r = 0.2

opti_cons1 = casadi.Opti();
x1 = opti_cons1.variable(3)

opti_cons1.minimize(var(x1))
opti_cons1.subject_to(dot(unit.T, x1) - 1 == 0)
opti_cons1.subject_to(dot(p.T, x1) - r1 == 0)

opti_cons1.set_initial(x1,x0)
opti_cons1.solver('ipopt')

print("\n \n  Résolution avec Casadi")
start_time = time.time()
sol = opti_cons1.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x1))
'''

'\n# équivalent avec Casadi pour r = 0.2\n\nopti_cons1 = casadi.Opti();\nx1 = opti_cons1.variable(3)\n\nopti_cons1.minimize(var(x1))\nopti_cons1.subject_to(dot(unit.T, x1) - 1 == 0)\nopti_cons1.subject_to(dot(p.T, x1) - r1 == 0)\n\nopti_cons1.set_initial(x1,x0)\nopti_cons1.solver(\'ipopt\')\n\nprint("\n \n  Résolution avec Casadi")\nstart_time = time.time()\nsol = opti_cons1.solve()\nprint("Durée : ", time.time()-start_time)\nprint(sol.value(x1))\n'

In [10]:
'''
# équivalent avec Casadi pour r = 0.15

opti_cons2 = casadi.Opti();
x2 = opti_cons2.variable(3)

opti_cons2.minimize(var(x2))
opti_cons2.subject_to(dot(unit.T, x2) - 1 == 0)
opti_cons2.subject_to(dot(p.T, x2) - r2 == 0)

opti_cons2.set_initial(x2,x0)
opti_cons2.solver('ipopt')

print("\n \n  Résolution avec Casadi")
start_time = time.time()
sol = opti_cons2.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x2))
'''

'\n# équivalent avec Casadi pour r = 0.15\n\nopti_cons2 = casadi.Opti();\nx2 = opti_cons2.variable(3)\n\nopti_cons2.minimize(var(x2))\nopti_cons2.subject_to(dot(unit.T, x2) - 1 == 0)\nopti_cons2.subject_to(dot(p.T, x2) - r2 == 0)\n\nopti_cons2.set_initial(x2,x0)\nopti_cons2.solver(\'ipopt\')\n\nprint("\n \n  Résolution avec Casadi")\nstart_time = time.time()\nsol = opti_cons2.solve()\nprint("Durée : ", time.time()-start_time)\nprint(sol.value(x2))\n'

On voit qu'une variation assez faible du rendement $r$ a une importance élevée sur la variance (on passe de $10^{-2}$ à $10^{-1}$ sur une variation assez faible de rendement, de l'ordre de quelques centièmes)

En revanche, on retrouve l'idée intuitive que pour un rendement d'investissement élevé (à prix équivalents), les allocations se concentrent davantage sur les actifs qui engendrent du bénéfice, mais donc le risque augmente aussi car on obtient des allocations réparties moins uniformément entre les différents actifs.